In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet18

# 1) EmotionCNN 정의
class EmotionCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1   = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2   = nn.Conv2d(32,64, 3, padding=1)
        self.pool    = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # [B,32,24,24]
        x = self.pool(F.relu(self.conv2(x)))  # [B,64,12,12]
        x = self.dropout(x)
        x = x.view(x.size(0), -1)             # flatten → [B, 64*12*12]
        return x

# 2) HybridEmotionModel 정의 (EmotionCNN + ResNet18 결합)
class HybridEmotionModel(nn.Module):
    def __init__(self):
        super().__init__()
        # (1) custom CNN 백본
        self.cnn = EmotionCNN()

        # (2) ResNet18 백본 (흑백 입력, 분류 레이어 제거)
        self.resnet = resnet18(weights=None)
        self.resnet.conv1 = nn.Conv2d(1, 64,
                                     kernel_size=7,
                                     stride=2,
                                     padding=3,
                                     bias=False)
        self.resnet.fc = nn.Identity()  # 최종 FC를 제거하고 feature만 반환

        # (3) 두 feature를 합친 뒤 감정 분류기
        self.fc = nn.Sequential(
            nn.Linear((64*12*12) + 512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 7)
        )

    def forward(self, x):
        # x1: EmotionCNN → [B, 9216],  x2: ResNet18 → [B, 512]
        x1 = self.cnn(x)
        x2 = self.resnet(x)
        x  = torch.cat((x1, x2), dim=1)  # [B, 9728]
        return self.fc(x)

# —— 위 정의 셀을 실행하신 뒤에는 아래 코드만 실행하세요 ——

# 3) Device 설정 (MPS 전용 예시)
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# 4) 모델 생성 및 MPS 이동
model = HybridEmotionModel().to(device)

In [ ]:

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# 1️⃣ 데이터 증강 (훈련용)
train_tf = transforms.Compose([
    transforms.Grayscale(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop(48, scale=(0.7, 1.0), ratio=(0.9, 1.1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 2️⃣ 검증용 전처리
val_tf = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 1) FER2013 로더
fer_train = ImageFolder('fer2013/train', transform=train_tf)
fer_val   = ImageFolder('fer2013/test',   transform=val_tf)
fer_train_loader = DataLoader(fer_train, batch_size=128, shuffle=True)
fer_val_loader   = DataLoader(fer_val, batch_size=128)

# 2) optimizer & criterion
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 500  # 원하는 만큼 조정 가능

for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for images, labels in fer_train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_train_loss = train_loss / len(fer_train_loader.dataset)
    train_acc = 100. * correct / total

    # —— Validation —— #
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    emotion_distribution = torch.zeros(7)  # FER2013은 7개 클래스

    with torch.no_grad():
        for images, labels in fer_val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            for p in preds:
                emotion_distribution[p.item()] += 1

    avg_val_loss = val_loss / len(fer_val_loader.dataset)
    val_acc = 100. * correct / total

    # —— 감정별 예측 분포 출력 —— #
    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    dist_str = ", ".join([
        f"{label}: {int(count)}" for label, count in zip(emotion_labels, emotion_distribution)
    ])

    print(f"[에폭 {epoch}/{num_epochs}] "
          f"훈련 손실: {avg_train_loss:.4f} | 훈련 정확도: {train_acc:.2f}% || "
          f"검증 손실: {avg_val_loss:.4f} | 검증 정확도: {val_acc:.2f}%")
    print(f"→ 예측 분포: {dist_str}\n")


In [ ]:
torch.save(model.state_dict(), "fer_pretrained.pth")
print("✅ FER2013 기반 모델이 fer_pretrained.pth로 저장되었습니다.")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet18
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
# 1️⃣ 데이터 증강 (훈련용)
# 1️⃣ RAF-DB 훈련 데이터 증강
raf_train_tf = transforms.Compose([
    transforms.Grayscale(),                          # 흑백 (모델 입력 채널에 맞춤)
    transforms.RandomHorizontalFlip(p=0.5),          # 좌우 반전
    transforms.RandomRotation(20),                   # ±20도 회전
    transforms.RandomResizedCrop(48, scale=(0.75, 1.0), ratio=(0.9, 1.1)),  # 크롭 + 확대 축소
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # 밝기 & 대비 변화
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 2️⃣ RAF-DB 검증 전처리
raf_val_tf = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


# 1) RAF-DB 로더
raf_train = ImageFolder('RAF_DB/train', transform=raf_train_tf)
raf_val   = ImageFolder('RAF_DB/test',   transform=raf_val_tf)
raf_train_loader = DataLoader(raf_train, batch_size=128, shuffle=True)
raf_val_loader   = DataLoader(raf_val, batch_size=128)

# 3) Device 설정 (MPS 전용 예시)
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# 4) 모델 생성 및 MPS 이동
model = HybridEmotionModel().to(device)

# 2) 기존 가중치 불러오기
model.load_state_dict(torch.load("fer_pretrained.pth"))

# 3) optimizer 재정의 (더 낮은 lr로)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

raf_epochs = 500

# 4) 파인튜닝 루프
# 4) 파인튜닝 루프
for epoch in range(1, raf_epochs + 1):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for x, y in raf_train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == y).sum().item()
        total += y.size(0)

    train_loss = running_loss / len(raf_train_loader.dataset)
    train_acc = 100. * correct / total

    # ——— 검증 ———
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    emotion_distribution = torch.zeros(7)

    with torch.no_grad():
        for x, y in raf_val_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)

            val_loss += loss.item() * x.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == y).sum().item()
            total += y.size(0)

            for p in preds:
                emotion_distribution[p.item()] += 1

    val_loss /= len(raf_val_loader.dataset)
    val_acc = 100. * correct / total

    # 감정별 분포 출력
    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    dist_str = ", ".join([
        f"{label}: {int(cnt)}" for label, cnt in zip(emotion_labels, emotion_distribution)
    ])

    print(f"[에폭 {epoch}/{raf_epochs}] "
          f"훈련 손실: {train_loss:.4f} | 훈련 정확도: {train_acc:.2f}% || "
          f"검증 손실: {val_loss:.4f} | 검증 정확도: {val_acc:.2f}%")
    print(f"→ 예측 분포: {dist_str}\n")

print("✅ RAF-DB 파인튜닝 완료.")
torch.save(model.state_dict(), "raf_finetuned.pth")
